In [1]:
import os

In [14]:
import json

In [15]:
from PIL import Image

In [16]:
import threading

In [17]:
import requests

In [2]:
client_id = "84f6a6e08e6e43b"

In [21]:
import os
import json
import requests
from PIL import Image


def get_imgur_client_id():
    """
     gets the imgur client id key from config.txt file.
    """
    with open('config.txt', 'r') as f:
        client_id = f.read()

    return client_id

def create_download_dir():
    """
    creates a download directory for images.
    """
    dir_images = os.path.join('images')

    if not os.path.exists(dir_images):
        os.mkdir(dir_images)

    return dir_images


def download_image_from_url(url, directory):
    """
    download image and save into given directory.
    """
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        filename = os.path.basename(url)
        filepath = os.path.join(directory, f'{filename}')
        with open(filepath, 'wb') as f:
            f.write(response.content)
    

def build_link_list(client_id, num_of_images):
    """
    builds a list of image links.
    """
    i = 1
    cnt = 0
    url_list = []
    url_list_len = []

    try:
        while(cnt < num_of_images):
            # get request
            response = requests.get(
                f'https://api.imgur.com/3/gallery/random/random/{i}', 
                headers={'Authorization': f'Client-ID {client_id}'},
                stream=True
            )
            
            # control
            if response.status_code == 200:
                data_list = json.loads(response.content)['data']
                url_list.extend([
                    i['link']
                    for i in data_list 
                    if 'type' in i 
                    and i['type'] in ('image/png', 'image/jpeg')
                    and i['link'] not in url_list
                ])

                cnt = len(url_list)
                url_list_len.append(cnt)
                i += 1
                
                # control if api doesn't return anything new
                if set(url_list_len[-10:]) == 1:
                    break
            
            elif response.status_code == 429:
                print('too many requests, enough, or you can choose to put time.sleep() in here...') 
                break

            else:
                break

    except:
        print('api limit reached!')
        
    
    return url_list


def create_thumbnail(size, path):
    """
    create resized version of the image path given, with the same name 
    extended with _thumbnail.
    """
    try:
        # create thumbnail
        image = Image.open(path)
        image.thumbnail(size)

        # create path for thumbnail
        dir_images = os.path.join(path)
        filename, extension = os.path.splitext(path)
        new_filename = os.path.join('{}{}{}'.format(filename, '_thumbnail', extension))

        # save thumbnail
        image.convert('RGB').save(new_filename)
    except:
        'image error'

In [7]:
NUM_OF_IMAGES = 1000 # max requests can be done per day is 12500

IMAGES_DIR = create_download_dir()

In [8]:
%%time

image_links = build_link_list(client_id, NUM_OF_IMAGES)
for image_link in image_links:
    download_image_from_url(image_link, IMAGES_DIR)

too many requests, enough, or you can choose to put time.sleep() in here...
CPU times: user 21.5 s, sys: 2.98 s, total: 24.5 s
Wall time: 1min 28s


In [25]:
len(image_links)

731

In [9]:
%%time

import threading

image_links = build_link_list(CLIENT_ID, NUM_OF_IMAGES)

def slave():
    for image_link in image_links:
        download_image_from_url(image_link, IMAGES_DIR)
threadings_list = []
for i in range(10):
    work = threading.Thread(target=slave)
    threadings_list.append(work)
    work.start()

too many requests, enough, or you can choose to put time.sleep() in here...
Done
CPU times: user 3.21 s, sys: 204 ms, total: 3.41 s
Wall time: 11.2 s


In [10]:
%%time

image_path_list = os.listdir('images')

for image_path in image_path_list:
    create_thumbnail((128, 128), os.path.join('images', image_path))

CPU times: user 19.9 s, sys: 2.98 s, total: 22.9 s
Wall time: 25.2 s


In [19]:
%%time

import multiprocessing
from multiprocessing import Pool
from itertools import product

p = multiprocessing.Pool(processes=9)

image_path_list = os.listdir('images')

for image_path in image_path_list:
    
    p.starmap(create_thumbnail, product((128, 128), os.path.join('images', image_path)))
    


CPU times: user 6.54 s, sys: 1.2 s, total: 7.75 s
Wall time: 9.68 s


In [23]:
from prettytable import PrettyTable
x = PrettyTable()

In [24]:
x.field_names = ["Description", "Time Taken"]
x.add_row(["John downloaded images", "1min 28s"])
x.add_row(["Download images", "11.2 s"])
x.add_row(["John resized images", "25.2 s"])
x.add_row(["MultiProcessing after resizing", "9.68 s"])
print(x)

+--------------------------------+------------+
|          Description           | Time Taken |
+--------------------------------+------------+
|     John downloaded images     |  1min 28s  |
|        Download images         |   11.2 s   |
|      John resized images       |   25.2 s   |
| MultiProcessing after resizing |   9.68 s   |
+--------------------------------+------------+
